# 28. El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones (**)

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
import os

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
contents_ori = sqlContext.read.parquet('Wikipedia dataset 2c2021/contents.parquet').rdd
contents_ori.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [8]:
contents = contents_ori.map(lambda x: ((x[6] if x[6] is not None else x[8]),(0 if x[9] is None else 1, 1)))
contents.take(2)

[('Metrónomo', (1, 1)), ('2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', (1, 1))]

In [10]:
contents = contents.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
contents.take(2)

[('Arctos71', (9, 47)), ('InternetArchiveBot', (61756, 61756))]

## == 1

In [43]:
solo1 = contents.filter(lambda x: x[1][1] == 1)
solo1.take(2)

[('45.71.202.51', (0, 1)), ('81.0.38.70', (1, 1))]

In [47]:
comentados_solo1 = solo1.map(lambda x: x[1][0]).reduce(lambda a, b: a+b)
editados_solo1 = solo1.map(lambda x: x[1][1]).reduce(lambda a, b: a+b)
porcentage_solo1 = ((editados_solo1 - comentados_solo1)/editados_solo1)*100
porcentage_solo1

35.21556458220743

## > 10

In [45]:
mayor10 = contents.filter(lambda x: x[1][1] > 10)
mayor10.take(2)

[('Arctos71', (9, 47)), ('InternetArchiveBot', (61756, 61756))]

In [48]:
comentados_mayor10 = mayor10.map(lambda x: x[1][0]).reduce(lambda a, b: a+b)
editados_mayor10 = mayor10.map(lambda x: x[1][1]).reduce(lambda a, b: a+b)
porcentage_mayor10 = ((editados_mayor10 - comentados_mayor10)/editados_mayor10)*100
porcentage_mayor10

10.140649236405128

## > 100

In [50]:
mayor100 = contents.filter(lambda x: x[1][1] > 100)
mayor100.take(2)

[('InternetArchiveBot', (61756, 61756)), ('Telephone Directory', (12, 229))]

In [51]:
comentados_mayor100 = mayor100.map(lambda x: x[1][0]).reduce(lambda a, b: a+b)
editados_mayor100 = mayor100.map(lambda x: x[1][1]).reduce(lambda a, b: a+b)
porcentage_mayor100 = ((editados_mayor100 - comentados_mayor100)/editados_mayor100)*100
porcentage_mayor100

9.365573554956518